### 1단계: 라이브러리 설치 및 불러오기

In [ ]:
# 필수 라이브러리 설치 
!pip install mlxtend

# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 로드 완료")

### 2단계: 데이터 로드 및 전처리

In [4]:
# 데이터 로드
df_clean = pd.read_csv('../data/cleaned/cleaned_data.csv')
print(f"정제 전 데이터 크기: {df_clean.shape}")
# print(df_clean.head())
# print(df_clean.info())

# 데이터 정제
# 1. 결측치 제거
df_clean = df_clean.dropna(subset=['InvoiceNo', 'StockCode', 'Description', 'CustomerID'])

# 취소 주문은 실제 구매 패턴이 아니므로 제외
# 2. 취소 주문 제거 (InvoiceNo가 'C'로 시작하는 경우)
df_clean = df_clean[~df_clean['InvoiceNo'].astype(str).str.startswith('C')]

#음수 수량/금액은 반품이므로 제외
# 3. 음수 수량 제거
df_clean = df_clean[df_clean['Quantity'] > 0]
# 4. 음수 금액 제거
df_clean = df_clean[df_clean['UnitPrice'] > 0]

# 5. 상품이 아닌 항목 제거 (배송비, 할인 등 - StockCode에 특수문자 포함)
df_clean = df_clean[df_clean['StockCode'].str.match(r'^[0-9]+[A-Z]*$')]

print(f"정제 후 데이터 크기: {df_clean.shape}")
print(f"전체 주문 수: {df_clean['InvoiceNo'].nunique()}")
print(f"전체 상품 수: {df_clean['StockCode'].nunique()}")

정제 전 데이터 크기: (541906, 8)
정제 후 데이터 크기: (396337, 8)
전체 주문 수: 18402
전체 상품 수: 3659


### 3단계: 데이터 트랜잭션 형식으로 변환
- 기준: 각 invoice에 어떤 상품이 포함되어있는가

In [5]:
# 3-1. 주문별 상품 리스트 : 각 Invoice에 포함된 상품명(Description) 리스트 생성
basket = df_clean.groupby('InvoiceNo')['Description'].apply(list).reset_index()

# print(basket.head())
print(f"\n전체 트랜잭션(주문) 수: {len(basket)}")

# 3-2. One-Hot 인코딩 (0/1 행렬 변환)
te = TransactionEncoder()
basket_encoded = te.fit_transform(basket['Description'])
basket_df = pd.DataFrame(basket_encoded, columns=te.columns_)

# print(basket_df.head())
print(basket_df.shape)


전체 트랜잭션(주문) 수: 18402
(18402, 3871)


### 4단계: 자주 함께 구매되는 상품 조합을 찾기(Apriori 알고리즘)

In [6]:
# 최소 지지도 설정 (너무 낮으면 조합이 너무 많고, 너무 높으면 의미 있는 조합 놓칠 수 있음)
min_support = 0.02  # 전체 주문의 1% 이상에서 나타나는 상품 조합을 찾음 -> 2%로 수정

# Apriori 알고리즘 실행
frequent_itemsets = apriori(basket_df, min_support=min_support, use_colnames=True)

print(f"빈발 항목 집합 개수: {len(frequent_itemsets)}")
print("\n상위 10개 빈발 항목:")
print(frequent_itemsets.sort_values('support', ascending=False).head(10))

빈발 항목 집합 개수: 242

상위 10개 빈발 항목:
      support                              itemsets
191  0.107108  (WHITE HANGING HEART T-LIGHT HOLDER)
153  0.092544            (REGENCY CAKESTAND 3 TIER)
81   0.086947             (JUMBO BAG RED RETROSPOT)
124  0.074938                       (PARTY BUNTING)
11   0.074720       (ASSORTED COLOUR BIRD ORNAMENT)
102  0.069992             (LUNCH BAG RED RETROSPOT)
165  0.062276   (SET OF 3 CAKE TINS PANTRY DESIGN )
95   0.057168             (LUNCH BAG  BLACK SKULL.)
116  0.055918     (PACK OF 72 RETROSPOT CAKE CASES)
179  0.054505                      (SPOTTY BUNTING)


- 첫 시도 문제 발생(지지도 1%, 집합 개수 980)
    - 실행 시간 139초 소요 -> 모델 튜닝 필요
        - 방법1. apriori 그대로 사용 -> 최소 지지도 수정(올리기)
        - 방법2. 알고리즘 fpgrowth로 바꾸기
        - 방법3. 상품 수 줄이기 (너무 안 팔린 상품은 제거)
- 두 번째 시도 (**지지도 2%로 수정**)
    - 실행 시간 5초로 단축, 집합 개수도 242개로 줄음

### 5단계: 연관 규칙 생성 및 필터링

In [7]:
# 5-1. 연관 규칙 생성

# metric: 'lift' 기준으로 규칙 생성
# min_threshold: Lift가 1.0보다 큰 규칙만 선택 (양의 상관관계)
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)

print(f"생성된 연관 규칙 수: {len(rules)}")
# rules.head()

생성된 연관 규칙 수: 76


- **출력 컬럼**
    - antecedents: 선행 상품 A (A를 샀을 때)
    - consequents: 후행 상품 B (B를 살 확률)
    - support: 전체 거래 중 A와 B를 함께 산 비율
    - confidence: A를 샀을 때 B도 살 확률
    - lift: A와 B의 연관성 강도

In [8]:
# 5-2. 의미 있는 규칙 필터링

# 필터링 조건
# 1. Lift > 2: 연관성이 강한 규칙만
# 2. Confidence > 0.3: 신뢰도 30% 이상
# 3. Support > 0.01: 최소 1% 이상의 거래에서 나타남

filtered_rules = rules[
    (rules['lift'] > 2) &
    (rules['confidence'] > 0.3) &
    (rules['support'] > 0.01)
].sort_values('lift', ascending=False)

print(f"필터링 후 규칙 수: {len(filtered_rules)}")
# filtered_rules.head(20)

# 5-3. DataFrame으로 정리
# 더 읽기 쉬운 형태로 변환
rules_summary = filtered_rules.copy()
rules_summary['antecedents'] = rules_summary['antecedents'].apply(lambda x: ', '.join(list(x)))
rules_summary['consequents'] = rules_summary['consequents'].apply(lambda x: ', '.join(list(x)))

# 주요 컬럼만 선택
rules_summary = rules_summary[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rules_summary = rules_summary.round(3)
# rules_summary.head(20)

# CSV로 저장(선택)
# rules_summary.to_csv('association_rules_top20.csv', index=False, encoding='utf-8-sig')

필터링 후 규칙 수: 68


### 6단계: 시각화
1. Support vs Confidence vs Lift 산점도
2. Top 10 연관 규칙 시각화
3. 네트워크 그래프

In [ ]:
plt.figure(figsize=(12, 5))

# Subplot 1: Support vs Confidence
plt.subplot(1, 2, 1)
plt.scatter(filtered_rules['support'], filtered_rules['confidence'], 
            c=filtered_rules['lift'], cmap='viridis', alpha=0.6, s=100)
plt.colorbar(label='Lift')
plt.xlabel('Support (지지도)', fontsize=12)
plt.ylabel('Confidence (신뢰도)', fontsize=12)
plt.title('Support vs Confidence (색상 = Lift)', fontsize=14)
plt.grid(True, alpha=0.3)

# Subplot 2: Support vs Lift
plt.subplot(1, 2, 2)
plt.scatter(filtered_rules['support'], filtered_rules['lift'], 
            c=filtered_rules['confidence'], cmap='coolwarm', alpha=0.6, s=100)
plt.colorbar(label='Confidence')
plt.xlabel('Support (지지도)', fontsize=12)
plt.ylabel('Lift (향상도)', fontsize=12)
plt.title('Support vs Lift (색상 = Confidence)', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('mba_scatter_plots.png', dpi=300, bbox_inches='tight')
plt.show()

**해석**
1. 지지도와 신뢰도에 따른 연관성 강도
   - 오른쪽 위:자주 팔리고,같이 살 확률도 높음 -> 가장 좋음
   - 왼쪽 위: Support 낮고 Confidence 높음 -> 함께 사는 경향만 강함
   - lift 색은 진한데 support가 낮음 -> 특정 상황에서만 강함
2. 거의 항상 같이 삼 -> 구조적으로 일관된 상품군 표현
   - 왼쪽 위: support 낮고 lift 높음 -> 거의 항상 같이 사지만 자주 팔리지 않음
   - 오른쪽 위로 갈수록 좋음 
   - confidence 색 진하면 lift도 높은 경향 ->구조적으로 일관된 상품군 표현

In [ ]:
top10_rules = filtered_rules.head(10).copy()

top10_rules['rule'] = (
    top10_rules['antecedents'].apply(lambda x: ', '.join(list(x))[:30]) +
    ' → ' +
    top10_rules['consequents'].apply(lambda x: ', '.join(list(x))[:30])
)

plt.figure(figsize=(12, 8))

bars = plt.barh(
    range(len(top10_rules)),
    top10_rules['lift'],
    color='skyblue'
)

# y축 눈금: 숫자만 (1, 2, 3 ...)
plt.yticks(
    range(len(top10_rules)),
    range(1, len(top10_rules) + 1)
)

# y축 이름
plt.ylabel('Rule', fontsize=14)

# 막대 위 + 시작점에서 텍스트 출력
for i, rule in enumerate(top10_rules['rule']):
    plt.text(
        0.2,          # 막대 시작점 근처
        i,            # y 위치
        rule,
        va='center',
        ha='left',
        fontsize=12,
        color='white',   
        fontweight='bold'
    )

plt.xlabel('Lift (향상도)', fontsize=14)
plt.title('Top 10 연관 규칙 (Lift 기준)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('top10_rules_lift.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# 네트워크 시각화를 위한 라이브러리 설치
!pip install networkx
import networkx as nx

# Top 15 규칙으로 네트워크 생성
top15_rules = filtered_rules.head(15)

# 그래프 생성
G = nx.DiGraph()
for idx, row in top15_rules.iterrows():
    antecedent = ', '.join(list(row['antecedents']))[:20]
    consequent = ', '.join(list(row['consequents']))[:20]
    G.add_edge(antecedent, consequent, weight=row['lift'])

# 시각화
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, k=0.5, iterations=50)

# 노드 그리기
nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='lightblue', alpha=0.9)

# 엣지 그리기 (두께는 lift에 비례)
edges = G.edges()
weights = [G[u][v]['weight'] for u, v in edges]
nx.draw_networkx_edges(G, pos, width=weights, alpha=0.6, 
                        edge_color='gray', arrows=True, arrowsize=20)

# 라벨 그리기
nx.draw_networkx_labels(G, pos, font_size=8, font_family='Malgun Gothic')

plt.title('상품 연관 규칙 네트워크 (Top 15)', fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.savefig('mba_network.png', dpi=300, bbox_inches='tight')
plt.show()


**해석**
- 같은 상품군끼리 클러스터 형성(ex.teacup계열)
- 서로 다른 상품군 간 연관은 약함 -> 카테고리 내부 구매 강함
- 크로스 카테고리 추천보다는 동일 테마/라인 내 업셀링이 효과적일 것으로 보임

In [35]:
# 인사이트 정리
# print("=" * 60)
# print(" 장바구니 분석 결과 요약")
# print("=" * 60)

# print(f"\n1. 전체 분석 대상:")
# print(f"   - 주문 수: {df_clean['InvoiceNo'].nunique():,}개")
# print(f"   - 상품 수: {df_clean['StockCode'].nunique():,}개")
# print(f"   - 고객 수: {df_clean['CustomerID'].nunique():,}명")

# print(f"\n2. 빈발 항목 집합:")
# print(f"   - 최소 지지도: {min_support * 100}%")
# print(f"   - 발견된 빈발 항목: {len(frequent_itemsets)}개")

# print(f"\n3. 연관 규칙:")
# print(f"   - 생성된 전체 규칙: {len(rules)}개")
# print(f"   - 필터링 후 의미 있는 규칙: {len(filtered_rules)}개")

# print(f"\n4. 주요 인사이트 (Top 5 규칙):")
# for idx, row in enumerate(filtered_rules.head(5).itertuples(), 1):
#     ant = ', '.join(list(row.antecedents))
#     cons = ', '.join(list(row.consequents))
#     print(f"\n   [{idx}] {ant}")
#     print(f"       → {cons}")
#     print(f"       Lift: {row.lift:.2f} | Confidence: {row.confidence:.1%} | Support: {row.support:.2%}")
#     print(f"       해석: '{ant}'를 구매한 고객은 일반 고객보다 {row.lift:.1f}배 더 '{cons}'를 구매함")
